# ESML - accelerator: Quick DEMO


In [1]:
import sys, os
sys.path.append(os.path.abspath("../azure-enterprise-scale-ml/esml/common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject
p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.inference_mode = False # We want "TRAIN" mode in this notebook

In [2]:
p.unregister_all_datasets(p.ws) # For DEMO purpose

Using GEN2 as Datastore


AttributeError: 'NoneType' object has no attribute 'get_default_keyvault'

In [23]:
#p.dev_test_prod = "dev"
p.describe()

Inference version: 1

 - ds01_diabetes
projects/project002/03_diabetes_model_reg/train/ds01_diabetes/in/dev/2021/01/01/
projects/project002/03_diabetes_model_reg/train/ds01_diabetes/out/bronze/dev/
projects/project002/03_diabetes_model_reg/train/ds01_diabetes/out/silver/dev/

 - ds02_other
projects/project002/03_diabetes_model_reg/train/ds02_other/in/dev/2021/01/01/
projects/project002/03_diabetes_model_reg/train/ds02_other/out/bronze/dev/
projects/project002/03_diabetes_model_reg/train/ds02_other/out/silver/dev/
 

Training GOLD (p.GoldPath)
projects/project002/03_diabetes_model_reg/train/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/03_diabetes_model_reg/inference/1/gold/dev/2021_06_21/9d6c3895a8af43f1ad4e6b75604b2168/
 

A)INFERENCE ONLINE: GOLD scored (example if realtime today)
projects/project002/03_diabetes_model_reg/inference/1/scored/dev/

from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws = Workspace.get(name = p.workspace_name,subscription_id = p.subscription_id,resource_group = p.resource_group,auth=auth)
ws.write_config(path=".", file_name="../../ws_config.json")

ws = Workspace.from_config("../ws_config.json") # Reads config.json 

# 2) ESML will Automap and Autoregister Azure ML Datasets - IN, SILVER, BRONZE, GOLD
- `Automap` and `Autoregister` Azure ML Datasets as: `IN, SILVER, BRONZE, GOLD`

In [24]:
from azureml.core import Workspace
ws, config_name = p.authenticate_workspace_and_write_config()
ws = p.get_workspace_from_config()
ws.name

'msft-weu-DEV-eap-proj02_ai-amls'

In [25]:
print("Are we in R&D state (no dataset versioning) = {}".format(p.rnd))

Are we in R&D state (no dataset versioning) = False


In [26]:
datastore = p.init(ws)

...
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  2021/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)

Inference mode (False = Training mode): False
Load data as Datasets....
ds01_diabetes
ds02_other

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 2 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002 

Dataset 'ds01_diabetes' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files
Dataset 'ds02_other' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


# 3) IN->`BRONZE->SILVER`->Gold
- Create dataset from PANDAS - Save to SILVER

In [27]:
import pandas as pd 
ds = p.DatasetByName("ds01_diabetes")
df = ds.Bronze.to_pandas_dataframe()
df.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


## 3) BRONZE-SILVER (EDIT rows & SAVE)
- Test change rows, same structure = new version (and new file added)
- Note: not earlier files in folder are removed. They are needed for other "versions". 
- Expected: For 3 files: New version, 997 rows: 2 older files=627 + 1 new file=370
- Expected (if we delete OLD files): New version, with less rows. 370 instead of 997

In [28]:
df_filtered = df[df.AGE > 0.015]
print(df.shape[0], df_filtered.shape[0])

442 185


## 3a) Save `SILVER` ds01_diabetes

In [29]:
aml_silver = p.save_silver(p.DatasetByName("ds01_diabetes"),df_filtered)
aml_silver.name

'M03_ds01_diabetes_train_SILVER'

### COMPARE `BRONZE vs SILVER`
- Compare and validate the feature engineering

In [30]:
ds01 = p.DatasetByName("ds01_diabetes")
bronze_rows = ds01.Bronze.to_pandas_dataframe().shape[0]
silver_rows = ds01.Silver.to_pandas_dataframe().shape[0]

print("Bronze: {}".format(bronze_rows)) # Expected 442 rows
print("Silver: {}".format(silver_rows)) # Expected 185 rows (filtered)

assert bronze_rows == 442,"BRONZE Should have 442 rows to start with, but is {}".format(bronze_rows)
assert silver_rows == 185,"SILVER should have 185 after filtering, but is {}".format(silver_rows)

Bronze: 442
Silver: 185


## 3b) Save  `BRONZE →  SILVER` ds02_other

In [31]:
df_edited = p.DatasetByName("ds02_other").Silver.to_pandas_dataframe()
ds02_silver = p.save_silver(p.DatasetByName("ds02_other"),df_edited)
ds02_silver.name

'M03_ds02_other_train_SILVER'

## 3c) Merge all `SILVERS -> then save GOLD`

In [32]:
df_01 = ds01.Silver.to_pandas_dataframe()
df_02 = ds02_silver.to_pandas_dataframe()
df_gold1_join = df_01.join(df_02) # left join -> NULL on df_02
print("Diabetes shape: ", df_01.shape)
print(df_gold1_join.shape)

Diabetes shape:  (185, 11)
(185, 19)


# Save `GOLD` v1

In [33]:
p.rnd=False # Allow versioning on DATASETS, to have lineage

In [34]:
ds_gold_v1 = p.save_gold(df_gold1_join)

### 3c) Ops! "faulty" GOLD - too many features

In [35]:
print(p.Gold.to_pandas_dataframe().shape) # 19 features...I want 11

(185, 19)


In [36]:
print("Are we in RnD phase? Or do we have 'versioning on datasets=ON'")
print("RnD phase = {}".format(p.rnd))

Are we in RnD phase? Or do we have 'versioning on datasets=ON'
RnD phase = False


# Save `GOLD` v2

In [37]:
# Lets just go with features from ds01
ds_gold_v1 = p.save_gold(df_01)

# Get `GOLD` by version

In [38]:
gold_1 = p.get_gold_version(1)
gold_1.to_pandas_dataframe().shape # (185, 19)

(185, 19)

In [39]:
gold_2 = p.get_gold_version(2)
gold_2.to_pandas_dataframe().shape # (185, 11)

(185, 11)

In [40]:
p.Gold.to_pandas_dataframe().shape # Latest version (185, 11)

(185, 11)

In [41]:
df_01_filtered = df_01[df_01.AGE > 0.03807]
ds_gold_v1 = p.save_gold(df_01_filtered)

In [42]:
gold_2 = p.get_gold_version(3) # sliced, from latest version
gold_2.to_pandas_dataframe().shape # (113, 11)

(113, 11)

# TRAIN - `AutoMLFactory + ComputeFactory`

In [43]:
from baselayer_azure_ml import AutoMLFactory, ComputeFactory

In [44]:
p.dev_test_prod = "test"
print("what environment are we targeting? =  {}".format(p.dev_test_prod)) 

what environment are we targeting? =  test


In [45]:
automl_performance_config = p.get_automl_performance_config()
automl_performance_config

Loading AutoML config settings from: test


{'enable_voting_ensemble': True,
 'enable_stack_ensemble': True,
 'model_explainability': True,
 'experiment_timeout_hours': 24,
 'iteration_timeout_minutes': 480,
 'n_cross_validations': 9,
 'enable_early_stopping': True,
 'iterations': 1000,
 'max_cores_per_iteration': -1,
 'allowed_models': ['LightGBM'],
 'blocked_models': ['GradientBoosting',
  'XGBoostRegressor',
  'ExtremeRandomTrees',
  'LassoLars',
  'RandomForest',
  'ElasticNet',
  'AutoArima'],
 'path': '.',
 'debug_log': 'azure_automl_debug_test.log'}

In [46]:
p.dev_test_prod = "dev"
automl_performance_config = p.get_automl_performance_config()
automl_performance_config

Loading AutoML config settings from: dev


{'enable_voting_ensemble': False,
 'enable_stack_ensemble': False,
 'model_explainability': True,
 'experiment_timeout_hours': 0.25,
 'iteration_timeout_minutes': 2,
 'n_cross_validations': 2,
 'enable_early_stopping': True,
 'iterations': 20,
 'max_cores_per_iteration': -1,
 'allowed_models': ['LightGBM'],
 'path': '.',
 'debug_log': 'azure_automl_debug_dev.log'}

# Get `COMPUTE` for current `ENVIRONMENT`

In [47]:
aml_compute = p.get_training_aml_compute(ws)

Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster prj02-m03-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = prj02-m03-dev


# `TRAIN` model -> See other notebook `esml_howto_2_train.ipynb`

In [3]:
from azureml.train.automl import AutoMLConfig
from baselayer_azure_ml import azure_metric_regression

label = "Y"
train_6, validate_set_2, test_set_2 = p.split_gold_3(0.6,label) # Auto-registerin AZURE (M03_GOLD_TRAIN | M03_GOLD_VALIDATE | M03_GOLD_TEST) # Alt: train,testv= p.Gold.random_split(percentage=0.8, seed=23)
automl_config = AutoMLConfig(task = 'regression',
                             primary_metric = azure_metric_regression.MAE,
                             compute_target = aml_compute,
                             training_data = p.GoldTrain, # is 'train_6' pandas dataframe, but as an Azure ML Dataset
                             label_column_name = label,
                             experiment_exit_score = '0.308', # DEMO purpose
                             **automl_performance_config
                            )

via_pipeline = False
best_run, fitted_model, experiment = AutoMLFactory(p).train_pipeline(automl_config) if via_pipeline else AutoMLFactory(p).train_as_run(automl_config)

AttributeError: 'NoneType' object has no attribute '_auth'

# END

# ESML - accelerator

## PROJECT + DATA CONCEPTS + ENTERPRISE Datalake Design + DEV->PROD MLOps
- `1)ESML Project`: The ONLY thing you need to remember is your `Project number` (and `BRONZE, SILVER, GOLD` concept )
    - ProjectNo=4 have a list of all your datasets as ESMLDatasets. (Well you need to provide names for them also: "mydata01", "mydata02" - but thats it)
- `2)Lakedesign & Roles`: Bronze, silver, gold + IN and date folders
    - Benefits: Physical datalake design!  onnected to Azure ML Workspace, with autoregistration of `Azure ML Datasets`
    - `Role 1`: `Data ingestion team` only need to care about 1 thing - onboard data to `IN-folder`, in .CSV format
        - `Auto parquet-conversion` from `IN` folder (.CSV) to `OUT`/BRONZE/bronze.PARQUET 
    - `Role 2`: `Data scientists` only need to care about 3 things (R/W): `BRONZE, SILVER, GOLD` datasets, all in .PARQUET format
    - How? The ESML project will `Automap` and `Autoregister` Azure ML Datasets - `IN, SILVER, BRONZE, GOLD`
- `2a) R&D  VS Production phase`: "Latest data" VS versioning on Datasets and datefolders  
    - Benefits "R&D mode": Faster RnD phase to onboard and refresh data easy. Also fast "flip-switch" to production
    - How? `ESMLDataset is context self aware` - knows when it is used in TRAIN or INFERENCE pipeline
- `2b) TRAIN vs INFERENCE` versions</u> `Reuse (Bronze->Silver->Gold) pipepline`, for both TRAIN preprocessing, and INFERENCE 
    - Benefits: Inference with different MODEL version, on data from the same day/time, (to compare scoring etc)
    - How? ESMLDataset have context self awareness, and `knows WHERE and HOW to load/save data`
- `2c) BATCH CONFIG`: Turn on/off features on ALL datasets
    - Accelerate setup: `Datadrift, Time series traits, Smart noise, etc`
    - Share refined data back to its "origin/non-projectbased structure" easy: 
        - ESMLProject.ShareBack(ds.Silver)
    - How? ESMProject controls all ESMDatasets, in a uniform way
## ENTERPRISE Deployment of Models & Governance - MLOps  at scale
- `3) DEV->TEST-PROD` (configs, compute, performance)
    - ESML has config for 3 environemnts: Easy DEPLOY model across subscriptions and Azure ML Studio workspaces 
        - Save costs & time: 
            - `DEV` has cheaper compute performance for TRAIN and INFERENCE (batch, AKS)
            - `DEV` has Quick-debug ML training (fast training...VS good scoring in TEST and PROD)
        - How? ESML `AutoMLFactory` and `ComputeFactory`
         

### Q&A:
- Q: Is ESML Machine learning specific? If I only want to refine some data...for integration, or report? 
- A: You can use this for just data refinement also: `Bronze->Silver->Gold` refinement.
    - Benefits: Enterprise security, Read/write to datalake, easy to share refined data. 
    - Benefits: The tooling "glued togehter": Azure datafactory +  Azure Databricks (and Azure ML Studio pipelines if needed)



In [29]:
p.dev_test_prod

'dev'

In [30]:
from baselayer_azure_ml import AutoMLFactory
target_env = p.dev_test_prod #"dev", test, prod  = Target environment. Does Model A score better than Model B?
print("Example: If new model scores better in DEV, we can promote this to TEST")

promote, m1_name, r1_id, m2_name, r2_run_id = AutoMLFactory(p).compare_scoring_current_vs_new_model(target_env)

print("Promote model?  {}".format(promote))
print("New Model: {} in environment {}".format(m1_name, p.dev_test_prod))
print("Existing Model: {} in environment {}".format(m2_name,target_env))

if (promote and p.dev_test_prod == target_env):# Can only register a model in same workspace (test->test) - need to retrain if going from dev->test
    AutoMLFactory(p).register_active_model(target_env)


Example: If new model scores better in DEV, we can promote this to TEST
Loading AutoML config settings from: dev
targe=source environement. Compare model version in DEV/TEST/PROD with latest registered in same DEV/TEST/PROD workspace (same workspace & subscriptiom comparison)
Package:azureml-automl-runtime, training version:1.27.0.post2, current version:1.26.0
Package:azureml-core, training version:1.27.0, current version:1.26.0
Package:azureml-dataprep, training version:2.14.2, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.12.1, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.27.0, current version:1.26.0
Package:azureml-defaults, training version:1.27.0, current version:1.26.0
Package:azureml-interpret, training version:1.27.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.27.0, current version:1.26.0
Package:azureml-telemet

In [31]:
inference_config, model, best_run = p.get_active_model_inference_config(ws) #  AutoML support 
service,api_uri, kv_aks_api_secret= p.deploy_automl_model_to_aks(model,inference_config)

Loading AutoML config settings from: dev
Loading AutoML config settings from: dev
Package:azureml-automl-runtime, training version:1.27.0.post2, current version:1.26.0
Package:azureml-core, training version:1.27.0, current version:1.26.0
Package:azureml-dataprep, training version:2.14.2, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.12.1, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.27.0, current version:1.26.0
Package:azureml-defaults, training version:1.27.0, current version:1.26.0
Package:azureml-interpret, training version:1.27.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.27.0, current version:1.26.0
Package:azureml-telemetry, training version:1.27.0, current version:1.26.0
Package:azureml-train-automl-client, training version:1.27.0, current version:1.26.0
Package:azureml-train-automl-runtime, training version:1.2

In [32]:
X_test, y_test, tags = p.get_gold_validate_Xy() # Get the X_test data, ESML knows the SPLIT and LABEL already (due to training)
print(tags)

df = p.call_webservice(p.ws, X_test,"my_caller_id_tracker_guid") # Auto-fetch key from keyvault, and calls the webservice
df.head()

M03_GOLD_VALIDATE : (23, 11)
X_test  (23, 10)
y_test  (23,)
{'split_percentage': '0.2', 'label': 'Y'}
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Note: Fetching keys automatically via workspace keyvault.
Saving scoring to lake for project folder project002 and inference_model_version: 1 ...
...

Saved DATA to score successfully in LAKE, as file 'to_score_my_caller_id_tracker_guid.parquet'
Saved SCORED data in LAKE, as file 'scored_my_caller_id_tracker_guid.parquet'


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,result
0,0.08,0.05,-0.00,-0.03,0.04,0.06,-0.01,0.03,-0.00,-0.00,143.96
1,0.05,0.05,0.06,0.03,0.03,-0.05,-0.05,0.07,0.13,0.14,236.46
2,0.07,-0.04,-0.01,-0.01,0.09,0.10,0.01,0.03,-0.01,0.03,118.17
3,0.05,0.05,-0.01,0.05,0.05,-0.02,-0.01,0.03,0.12,-0.02,177.14
4,0.05,0.05,-0.01,-0.03,-0.01,0.00,-0.04,0.03,0.03,0.03,168.26
